## How to simply use tf.data
> tf.data package를 사용하는 방법에 관한 예시, 예시 데이터는 numpy package를 이용하여 간단하게 data에 해당하는 X, target에 해당하는 y를 생성하여 tf.data package의 각종 module, function을 이용한다. epoch 마다 validation data에 대해서 validation을 하는 상황을 가정

### Template
> for문을 활용, model을 training시 data pipeline으로써 아래의 function과 method를 사용하는 방법에 대한 예시

- Dataset class 
    + tf.data.Dataset.from_tensor_slices으로 Dataset class의 instance를 생성
        * train data에 대한 Dataset class의 instance, tr_data
        * validation data에 대한 Dataset class의 instance, val_data
    + 아래와 같은 method를 활용하여 training 시 필요한 요소를 지정 
        * instance의 shuffle method를 활용하여 shuffling
        * instance의 batch method를 활용하여 batch size 지정 
        * for문으로 전체 epoch를 control하므로 repeat method는 활용하지 않음 
        
<br>
- Iterator class
    + Dataset class의 instance에서 make_initializable_iterator method로 iterator class의 instance를 생성 
        * train data에 대한 iterator class의 instance, tr_iterator
        * validation data에 대한 iterator class의 instance. val_iterator
        * make_initializable_iterator()로 iterator class의 instance를 생성할 경우, random_seed를 고정x 
            - random_seed를 고정할 경우, 서로 다른 epoch의 step별 mini-batch의 구성이 완전히 똑같아지기 때문
    + Anonymous iterator를 tf.data.iterator.from_string_handle로 생성
        * string_handle argument에 tf.placeholder를 이용
            - tr_iterator를 활용할 것인지, val_iterator를 활용할 것인지 조절 
            
            <br>
- Reference
  + [aisolab, 'How to simply use tf.data'](https://github.com/aisolab/CS20/blob/master/Lec03_Linear%20and%20Logistic%20Regression/How%20to%20simply%20use%20tf.data.ipynb)

## when use tf.data?
- For prototype, feed dict can be faster and easier to write
- tf.data is tricky to use when you have complicated preprocessing or multiple data sources
- NLP data is normally just a sequence of integers. In this case, transferring the data over to GPU is pretty quick, so the speedup of tf.data isn;t that large

### Setup

In [1]:
import numpy as np
import tensorflow as tf

print(tf.__version__)

1.11.0


In [3]:
# 전체 데이터의 개수가 12개인 임의의 데이터셋 생성
x = np.c_[np.arange(12), np.arange(12)]
y = np.arange(12)

print(x.shape, y.shape)
print(x)

(12, 2) (12,)
[[ 0  0]
 [ 1  1]
 [ 2  2]
 [ 3  3]
 [ 4  4]
 [ 5  5]
 [ 6  6]
 [ 7  7]
 [ 8  8]
 [ 9  9]
 [10 10]
 [11 11]]


In [4]:
# 위의 데이터를 train, validation으로 split
x_tr = x[:8]
y_tr = y[:8]

x_val = x[8:]
y_val = y[8:]

print(x_tr.shape, y_tr.shape)
print(x_val.shape, y_val.shape)

(8, 2) (8,)
(4, 2) (4,)


### Template

In [5]:
n_epoch = 3
batch_size = 2
total_steps = int(x.shape[0] / batch_size)

print('epoch: {}, batch_size: {}, total_steps: {}'.format(n_epoch, batch_size, total_steps))

epoch: 3, batch_size: 2, total_steps: 6


In [7]:
tr_data = tf.data.Dataset.from_tensor_slices((x_tr, y_tr))
tr_data = tr_data.shuffle(buffer_size=30)
tr_data = tr_data.batch(batch_size=batch_size)

val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_data = val_data.batch(batch_size=batch_size)

print(tr_data)
print(val_data)

<BatchDataset shapes: ((?, 2), (?,)), types: (tf.int32, tf.int32)>
<BatchDataset shapes: ((?, 2), (?,)), types: (tf.int32, tf.int32)>


In [8]:
tr_iterator = tr_data.make_initializable_iterator()
val_iterator = val_data.make_initializable_iterator()

handle = tf.placeholder(dtype=tf.string)

In [11]:
iterator = tf.data.Iterator.from_string_handle(string_handle=handle,
                                              output_shapes=tr_iterator.output_shapes,
                                              output_types=tr_iterator.output_types)
x_mb, y_mb = iterator.get_next()

In [16]:
# n_tr_step, n_val_step 변수와 관련된 코드는 step수 확인을 위해 넣음
sess = tf.Session()
tr_handle, val_handle = sess.run([tr_iterator.string_handle(), val_iterator.string_handle()])

for epoch in range(n_epoch):
    print('epoch: {} training start'.format(epoch+1))
    sess.run(tr_iterator.initializer) # Run tr_iterator
    n_tr_step = 0
    while True:
        try:
            n_tr_step += 1
            X_tmp, y_tmp = sess.run([x_mb, y_mb], feed_dict = {handle : tr_handle})
            print('step: {}'.format(n_tr_step))
            print(x_tmp, y_tmp)
        except:
            print('epoch: {} training finished'.format(epoch+1))
            break
    
    print('at epoch: {}, validation start'.format(epoch+1))
    sess.run(val_iterator.initializer)
    n_val_step = 0
    while True:
        try:
            n_val_step += 1
            x_tmp, y_tmp = sess.run([x_mb, y_mb], feed_dict={handle: val_handle})
            
            print('step: {}'.format(n_val_step))
            print(x_tmp, y_tmp)
        except:
            print('validation finished')
            break

epoch: 1 training start
step: 1
[[10 10]
 [11 11]] [5 7]
step: 2
[[10 10]
 [11 11]] [0 1]
step: 3
[[10 10]
 [11 11]] [3 6]
step: 4
[[10 10]
 [11 11]] [2 4]
epoch: 1 training finished
at epoch: 1, validation start
step: 1
[[8 8]
 [9 9]] [8 9]
step: 2
[[10 10]
 [11 11]] [10 11]
validation finished
epoch: 2 training start
step: 1
[[10 10]
 [11 11]] [4 2]
step: 2
[[10 10]
 [11 11]] [1 7]
step: 3
[[10 10]
 [11 11]] [6 0]
step: 4
[[10 10]
 [11 11]] [3 5]
epoch: 2 training finished
at epoch: 2, validation start
step: 1
[[8 8]
 [9 9]] [8 9]
step: 2
[[10 10]
 [11 11]] [10 11]
validation finished
epoch: 3 training start
step: 1
[[10 10]
 [11 11]] [6 1]
step: 2
[[10 10]
 [11 11]] [4 0]
step: 3
[[10 10]
 [11 11]] [5 7]
step: 4
[[10 10]
 [11 11]] [3 2]
epoch: 3 training finished
at epoch: 3, validation start
step: 1
[[8 8]
 [9 9]] [8 9]
step: 2
[[10 10]
 [11 11]] [10 11]
validation finished
